# 0. Justificación de uso de Spotify para sacar toptracks.

# 1. Sacar datos Spotify de 5 años, por países. 

In [94]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [95]:
# Coloca tus credenciales aquí
CLIENT_ID = '1a0e65e89f324cf2a3cf6d823d3e07ac'
CLIENT_SECRET = '86f0c0ff67af4653b59a2cc4b8f2d6ae'
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

Aquí está el diccionario de países por el que hay que iterar. Sacaremos cada una 4 df, uno por cada grupo de países. 

In [97]:
paises_mediterraneos = {
    "ES": "España",
    "FR": "Francia",
    "MC": "Mónaco",
    "IT": "Italia",
    "SI": "Eslovenia",
    "HR": "Croacia",
    "BA": "Bosnia y Herzegovina",
    "ME": "Montenegro",
    "AL": "Albania",
    "GR": "Grecia",
    "MT": "Malta",
    "CY": "Chipre"}
paises_este_europa = {
    "BY": "Bielorrusia",
    "BG": "Bulgaria",
    "CZ": "Chequia",
    "SK": "Eslovaquia",
    "HU": "Hungría",
    "MD": "Moldavia",
    "PL": "Polonia",
    "RO": "Rumanía",
    "UA": "Ucrania"}
paises_norte_europa = {
    "DK": "Dinamarca",
    "EE": "Estonia",
    "FI": "Finlandia",
    "IS": "Islandia",
    "IE": "Irlanda",
    "LT": "Lituania",
    "LV": "Letonia",
    "NO": "Noruega",
    "SE": "Suecia",
    "GB": "Reino Unido"}
paises_europa_central = {
    "AT": "Austria",
    "DE": "Alemania",
    "CH": "Suiza",
    "LI": "Liechtenstein",
    "BE": "Bélgica",
    "LU": "Luxemburgo",
    "NL": "Países Bajos"}

In [98]:
datos_rango = sp.search(q = "year:2000-2004", limit=50, offset=0, type='track', market="ES")

Esta función sirve para ir sacando los datos para cada grupo de países. Habrá que hacer una llamada para cada grupo de países (cada diccionario de los de arriba). 

La función toma como argumentos el diccionario y el rango de años que nos interese en el formato `2000-2004`

In [99]:
def extraccion_por_paises(diccionario_zonas, años):
    todos_los_datos = [] # Esto es una lista de dicts donde se van guardando los diccionarios de cada país
    for pais in diccionario_zonas.keys():
        datos_rango = sp.search(q = f'year:{años}', limit=50, offset=0, type='track', market={pais}) # Cada una tiene que cambiar en q el rango de años
        datos_por_pais = {}
        datos_por_pais['pais'] = pais
        datos_por_pais['datos'] = datos_rango
        todos_los_datos.append(datos_por_pais)
    return todos_los_datos
   

Aquí dejamos un ejemplo de utilización de la función:

In [111]:
prueba_con_miedo = extraccion_por_paises(paises_europa_central, "2015-2019")

In [112]:
for element in prueba_con_miedo:
    print(element)

{'pais': 'AT', 'datos': {'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=50&query=year%3A2015-2019&type=track&market=AT', 'limit': 50, 'next': None, 'offset': 0, 'previous': None, 'total': 50, 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2YZyLoL8N0Wb9xBt1NhZWg'}, 'href': 'https://api.spotify.com/v1/artists/2YZyLoL8N0Wb9xBt1NhZWg', 'id': '2YZyLoL8N0Wb9xBt1NhZWg', 'name': 'Kendrick Lamar', 'type': 'artist', 'uri': 'spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/7tYKF4w9nC0nq9CsPZTHyP'}, 'href': 'https://api.spotify.com/v1/artists/7tYKF4w9nC0nq9CsPZTHyP', 'id': '7tYKF4w9nC0nq9CsPZTHyP', 'name': 'SZA', 'type': 'artist', 'uri': 'spotify:artist:7tYKF4w9nC0nq9CsPZTHyP'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/3pLdWdkj83EYfDN6H2N8MR'}, 'href': 'https://api.spotify.com/v1/albums/3pLdWdkj83EYfDN6H2N8MR', 'id': '3pLdWdkj83EY

## 1.1. Filtrar ahora los datos que hemos sacado

En el diccionario `prueba` es donde iremos metiendo los datos que extraigamos

In [115]:
diccionario = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}

Con el código de abajo se extraen los datos de spotify. Luego hacemos una función. 

La función `filtro_de_datos` toma como argumento la salida de la función `extraccion_por_paises`, que es una lista donde hemos ido almacenando las salidas de las llamadas. 

In [ ]:
for datos_pais in prueba_con_miedo:
    for element in datos_pais['datos']['tracks']['items']:
        diccionario['pais'].append(datos_pais['pais'])
        diccionario['popularity'].append(element['popularity'])
        diccionario['nombre'].append(element['name'])
        artistas = []
        for artista in element['artists']:
            artistas.append(artista['name'])   
        artistas_str = ', '.join(artistas)
        diccionario['artista'].append(artistas_str)

In [114]:
# Un intento de función con el bucle de arriba:

def filtro_de_datos(lista_datos_zonas, diccionario):
    for datos_pais in lista_datos_zonas:
        for element in datos_pais['datos']['tracks']['items']:
            diccionario['pais'].append(datos_pais['pais'])
            diccionario['popularity'].append(element['popularity'])
            diccionario['nombre'].append(element['name'])
            artistas = []
            for artista in element['artists']:
                artistas.append(artista['name'])   
            artistas_str = ', '.join(artistas)
            diccionario['artista'].append(artistas_str)

In [81]:
import pandas as pd

# 2. Sacar de Lastfm el género de las canciones. 

In [82]:
import requests

In [83]:
# Coloca tus credenciales de lastfm aquí
CLIENT_ID = ''
API_KEY = 'da05ed14f4238789a27874ae45ee6729'

last_fm_connect = requests.get(f"http://www.last.fm/api/auth/?api_key={API_KEY}")

Aquí hay algunas pruebas que hemos hecho para ver cómo sacar el género de una canción con la API de lastfm.

In [84]:

test_2 = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist=Sasha&track=here+she+comes+again&api_key={API_KEY}&autocorrect=1&format=json")

In [ ]:
test_2.json()['toptags']['tag'][0]['name'] # Nos quedaríamos con las primera porque es la que más se repite (count más alto)

'pop'

In [86]:
for coso in test_2.json()['toptags']['tag']:
    print(coso['name'])

pop
Love
german
5 of 10 stars
beautiful english ones
lts
edhits


🧠Recordatorio: Como la llamada test_2 nos devuelve varios géneros (tags), decidimos quedarnos con el primero, que es el más utilizado. 

Queremos extraer el género para introducirlo en el diccionario `prueba`, por lo que hacemos dos listas, una con los nombres de las canciones y otra con los artistas. Iremos iterando por las dos a la vez para poder hacer la llamada a la API de lastfm.

In [87]:
lista_nombres = list(prueba.values())[0]
lista_artistas = list(prueba.values())[2]

In [88]:
lista_nombres[0]

'All The Stars (with SZA)'

In [89]:
lista_artistas[0]

'Kendrick Lamar, SZA'

Con este bucle hacemos la llamada a la API. El try/except controla que el código no se pare cuando haya alguna llamada que nos de un error o nos devuelva una canción sin tags. 

In [90]:
def llamada_lastfm(lista_nombres, lista_artistas,datos):
    for i in range(len(lista_nombres)):
        nombre = str(lista_nombres[i]).replace(' ', '+')
        artista = lista_artistas[i].replace(' ', '+').replace(',', '+&')
        llamada_genero = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist={artista}&track={nombre}&api_key={API_KEY}&autocorrect=1&format=json")
        try:
            genero_cancion = llamada_genero.json()['toptags']['tag'][0]['name']
            prueba['genero'].append(genero_cancion)
        except KeyError:
            prueba['genero'].append("NULL")
            print(f"La canción {lista_nombres[i]} no está registrada en lastfm")
        
        except IndexError:
            prueba['genero'].append("NULL")
            print(f"La canción {lista_nombres[i]} no tiene tags")


Ahora que ya tenemos todo el diccionario `prueba` completo, lo convertimos a Data Frame.

In [91]:
import pandas as pd

In [92]:
MI_MARAVILLOSO_DF = pd.DataFrame(prueba)

ValueError: All arrays must be of the same length

In [ ]:
MI_MARAVILLOSO_DF.index.name="id"

In [ ]:
MI_MARAVILLOSO_DF

,nombre,popularity,artista,pais,genero
id,,,,,
0,Unwritten,81,Natasha Bedingfield,AT,pop
1,Angel,28,"Shaggy, Rayvon",AT,NULL
2,Mr. Brightside,82,The Killers,AT,rock
3,Rooftop,0,Sasha,AT,pop
4,Numb,83,Linkin Park,AT,rock
...,...,...,...,...,...
345,Mzael Kel El Banat,0,Wael Kfoury,NL,NULL
346,Ms. Jackson,78,Outkast,NL,Hip-Hop
347,Ergaa Lel Shoua,0,Elissa,NL,NULL


Por último, pasamos el Data Frame a csv. 

🧠 Recuerda cambiar el nombre del csv antes de ejecutar la celda.

In [ ]:
MI_MARAVILLOSO_DF.to_csv("files/europa_central_2000_2004.csv")

# 3. Crear la bbdd

# 4. Alimentar la bbdd

# 5. Next steps

- Sacar más de 50 canciones por llamadas